In [1]:
from sqlalchemy.sql import text
from sqlalchemy import create_engine
import pymssql
import pymysql
import names
import random
import time
import uuid

In [2]:
#firstname, lastname, phone number, email
class fakepiientry:

    def __init__(self):
        self.domains = ["hotmail.com", "gmail.com", "aol.com", "mail.com" , "mail.kz", "yahoo.com"]
        self.letters = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]
        self.genders = ["male", "female"]
        self.countries = ["CHN", "USA", "SOV", "JAP", "GER"]
        self.cities = ["New York", "Los Angeles", "San Francisco", "Boston", "Riverside", "Seattle"]
        self.stateprovinces = ["CA", "NY", "WA", "IL", "IN"]
        
    def generate_phonenumber(self):
        first = str(random.randint(100,999))
        second = str(random.randint(1,888)).zfill(3)
        last = (str(random.randint(1,9998)).zfill(4))
        while last in ['1111','2222','3333','4444','5555','6666','7777','8888']:
            last = (str(random.randint(1,9998)).zfill(4))
        return ('{}-{}-{}'.format(first,second, last))
    
    def generate_random_email(self):
        namelen = random.randint(10,20)
        one_name = str(''.join([self.letters[random.randint(0,len(self.letters)-1)] for i in range(namelen)]))         
        one_domain = str(self.domains[random.randint(0, len(self.domains)-1)])
        return(one_name  + "@" + one_domain)

    def generate_random_gender(self):
        return self.genders[random.randint(0, len(self.genders)-1)]
        
    def generate_random_country(self):
        return self.countries[random.randint(0, len(self.countries)-1)]
    
    def generate_random_city(self):
        return self.cities[random.randint(0, len(self.cities)-1)]
    
    def generate_random_stateprovince(self):
        return self.stateprovinces[random.randint(0, len(self.stateprovinces)-1)]

    def pii_2nd_entry_gen(self, uid):
        return (uid, names.get_first_name(), names.get_last_name(), self.generate_random_gender(), random.randint(0, 10), random.randint(10, 100))
    
    def pii_3rd_entry_gen(self, uid):
        return (uid, self.generate_phonenumber(), self.generate_random_email(), random.randint(0, 10), random.randint(10, 100))

    def pii_4th_entry_gen(self, uid):
        return (uid, self.generate_random_country(), self.generate_random_city(), self.generate_random_stateprovince(), random.randint(0, 10), random.randint(10, 100))
    
myfakepiientry = fakepiientry()
for i in range(100, 105):
    thispii2nd = myfakepiientry.pii_2nd_entry_gen(i)
    thispii3rd = myfakepiientry.pii_3rd_entry_gen(i)
    thispii4th = myfakepiientry.pii_4th_entry_gen(i)
    print(thispii2nd)
    print(thispii3rd)
    print(thispii4th)

(100, 'Richard', 'Davis', 'female', 7, 23)
(100, '159-548-8064', 'fpkggecufuyzn@mail.com', 10, 96)
(100, 'USA', 'Riverside', 'WA', 6, 86)
(101, 'James', 'Boom', 'female', 10, 44)
(101, '294-256-3981', 'jgrnmakrrxauckahcfw@aol.com', 2, 22)
(101, 'GER', 'New York', 'NY', 5, 49)
(102, 'Lamont', 'Granado', 'female', 0, 27)
(102, '141-725-5642', 'rocuvcontaghum@yahoo.com', 8, 69)
(102, 'USA', 'Los Angeles', 'IN', 2, 69)
(103, 'Jordon', 'Brown', 'male', 8, 75)
(103, '354-416-3845', 'lrgzvxhpvcppkkeimm@hotmail.com', 5, 92)
(103, 'SOV', 'Seattle', 'IL', 6, 56)
(104, 'Ada', 'Miller', 'male', 3, 60)
(104, '635-664-3923', 'qfpidxdzlazt@mail.com', 1, 53)
(104, 'GER', 'Riverside', 'WA', 1, 62)


In [3]:
#conn = pymssql.connect("192.168.7.74", "SA", "Helios12$", "huaPIITest00")
#c1 = conn.cursor()
#c1.execute("SELECT * FROM INFORMATION_SCHEMA.TABLES")
#c1.execute("SELECT * FROM pii_1st")
#print( c1.fetchall() )
#c1.execute("create table dbo.pii_1st (uid int PRIMARY KEY NOT NULL)")
#c1.execute("IF OBJECT_ID('pii_1st', 'U') IS NOT NULL DROP TABLE pii_1st")
#c1.execute("CREATE TABLE pii_1st (uid INT NOT NULL)")
#conn.commit()
#c1.execute("select * from INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='pii_1st'")
#for x in c1.fetchall():
#    print(x)
#conn.close()

In [4]:
class mssqldbprocess:

    def __init__(self, hostip, usrname, pwd, dbname):
        self.hostip = hostip
        self.usrname = usrname
        self.pwd = pwd
        self.dbname = dbname
    
    def createpii(self):
        conn = pymssql.connect(self.hostip, self.usrname, self.pwd, self.dbname)
        c1 = conn.cursor()
        # table 1 for uid
        c1.execute("IF OBJECT_ID('pii_1st', 'U') IS NOT NULL DROP TABLE pii_1st")
        c1.execute("CREATE TABLE pii_1st (uid INT NOT NULL, tab1col1 INT, tab1col2 INT)")
        c1.execute("IF OBJECT_ID('pii_2nd', 'U') IS NOT NULL DROP TABLE pii_2nd")
        c1.execute("CREATE TABLE pii_2nd (uid INT NOT NULL, firstname VARCHAR(100), lastname VARCHAR(100), gender VARCHAR(100), tab2col1 INT, tab2col2 INT)")
        c1.execute("IF OBJECT_ID('pii_3rd', 'U') IS NOT NULL DROP TABLE pii_3rd")
        c1.execute("CREATE TABLE pii_3rd (uid INT NOT NULL, phonenumber VARCHAR(100), email VARCHAR(100), tab3col1 INT, tab3col2 INT)")
        c1.execute("IF OBJECT_ID('pii_4th', 'U') IS NOT NULL DROP TABLE pii_4th")
        c1.execute("CREATE TABLE pii_4th (uid INT NOT NULL, country VARCHAR(100), city VARCHAR(100), stateprovince VARCHAR(100), tab4col1 INT, tab4col2 INT)")
        conn.commit()
        conn.close()
        return 
        
    def fillpii(self, uid_beg, uid_end):
        conn = pymssql.connect(self.hostip, self.usrname, self.pwd, self.dbname)
        
        c = conn.cursor()
        entries_1st = []
        entries_2nd = []
        entries_3rd = []
        entries_4th = []
        
        for i in range(uid_beg, uid_end):
            #print(i)
            entries_1st.append((i, random.randint(0, 10), random.randint(10, 100)))
            entries_2nd.append(myfakepiientry.pii_2nd_entry_gen(i))
            entries_3rd.append(myfakepiientry.pii_3rd_entry_gen(i))
            entries_4th.append(myfakepiientry.pii_4th_entry_gen(i))            
        
        c.executemany("INSERT INTO pii_1st VALUES (%d, %d, %d)", entries_1st)
        c.executemany("INSERT INTO pii_2nd VALUES (%d, %s, %s, %s, %d, %d)", entries_2nd)
        c.executemany("INSERT INTO pii_3rd VALUES (%d, %s, %s, %d, %d)", entries_3rd)
        c.executemany("INSERT INTO pii_4th VALUES (%d, %s, %s, %s, %d, %d)", entries_4th)
        
        conn.commit()
        conn.close()
        return

#mymssqldbprocess = mssqldbprocess("192.168.7.74", "SA", "Helios12$", "huaPIITest00")
mymssqldbprocess = mssqldbprocess("192.168.7.74", "SA", "Helios12$", "huaPIITest10")
#mymssqldbprocess = mssqldbprocess("192.168.7.75", "SA", "Helios12$", "huaPIITest01")
#mymssqldbprocess.createpii()
#batch_size = 50
#for i in range(0, 2):
#    mymssqldbprocess.fillpii(i*batch_size , (i+1)*batch_size)

In [5]:
#firstname, lastname, phone number, email
class fakeeduentry:

    def __init__(self):
        self.courses = ["PHYS210", "PHYS110", "CS231", "CS101", "MATH107", "MATH108"]
        self.evals = ["A", "B", "C", "D", "E", "F"]

    def generate_phonenumber(self):
        first = str(random.randint(100,999))
        second = str(random.randint(1,888)).zfill(3)
        last = (str(random.randint(1,9998)).zfill(4))
        while last in ['1111','2222','3333','4444','5555','6666','7777','8888']:
            last = (str(random.randint(1,9998)).zfill(4))
        return ('{}-{}-{}'.format(first,second, last))
    
    def generate_random_course(self):
        return self.courses[random.randint(0, len(self.courses)-1)]
    
    def generate_random_eval(self):
        return self.evals[random.randint(0, len(self.evals)-1)]


    #course, class
    def edu_2nd_entry_gen(self, uid):
        return (uid, names.get_first_name() + ' ' +  names.get_last_name(), self.generate_random_course())
    
    #eval, evaluation
    def edu_3rd_entry_gen(self, uid):
        return (uid, self.generate_random_course(), self.generate_random_eval())
    
    #registration, student
    def edu_4th_entry_gen(self, uid):
        return (uid, names.get_first_name() + ' ' +  names.get_last_name(), self.generate_phonenumber())
    
myfakeeduentry = fakeeduentry()
for i in range(100, 105):
    thisedu2nd = myfakeeduentry.edu_2nd_entry_gen(i)
    thisedu3rd = myfakeeduentry.edu_3rd_entry_gen(i)
    thisedu4th = myfakeeduentry.edu_4th_entry_gen(i)
    print(thisedu2nd)
    print(thisedu3rd)
    print(thisedu4th)

(100, 'Janet Bruhn', 'MATH108')
(100, 'CS231', 'F')
(100, 'Ruth Carney', '858-771-1630')
(101, 'Joseph Backlund', 'CS101')
(101, 'PHYS210', 'B')
(101, 'Debra Crabtree', '242-571-1718')
(102, 'Victor Decelles', 'MATH108')
(102, 'CS101', 'C')
(102, 'Edith Phillips', '538-076-6698')
(103, 'Robbie Butler', 'MATH107')
(103, 'CS101', 'F')
(103, 'Justin Pate', '638-578-6879')
(104, 'James Ellis', 'CS101')
(104, 'PHYS210', 'E')
(104, 'Stephanie Mccaslin', '846-664-0860')


In [11]:
class mssqldbprocess_edu:

    def __init__(self, hostip, usrname, pwd, dbname):
        self.hostip = hostip
        self.usrname = usrname
        self.pwd = pwd
        self.dbname = dbname
    
    def createedu(self):
        conn = pymssql.connect(self.hostip, self.usrname, self.pwd, self.dbname)
        c1 = conn.cursor()
        # table 1 for uid
        if self.dbname == "course":
            c1.execute("IF OBJECT_ID('class', 'U') IS NOT NULL DROP TABLE class")
            c1.execute("CREATE TABLE class (ssn INT NOT NULL, name VARCHAR(100), course VARCHAR(100))")
        elif self.dbname == "eval":
            c1.execute("IF OBJECT_ID('evaluation', 'U') IS NOT NULL DROP TABLE evaluation")
            c1.execute("CREATE TABLE evaluation (ssn INT NOT NULL, course VARCHAR(100), eval VARCHAR(100))")
        elif self.dbname == "registration":
            c1.execute("IF OBJECT_ID('student', 'U') IS NOT NULL DROP TABLE student")
            c1.execute("CREATE TABLE student (ssn INT NOT NULL, name VARCHAR(100), phonenumber VARCHAR(100))")
        conn.commit()
        conn.close()
        return 
        
    def filledu(self, uid_beg, uid_end):
        conn = pymssql.connect(self.hostip, self.usrname, self.pwd, self.dbname)
        
        c = conn.cursor()
        entries_2nd = []
        entries_3rd = []
        entries_4th = []
        
        for i in range(uid_beg, uid_end):
            #print(i)
            entries_2nd.append(myfakeeduentry.edu_2nd_entry_gen(i))
            entries_3rd.append(myfakeeduentry.edu_3rd_entry_gen(i))
            entries_4th.append(myfakeeduentry.edu_4th_entry_gen(i))            
        
        if self.dbname == "course":
            c.executemany("INSERT INTO class VALUES (%d, %s, %s)", entries_2nd)
        elif self.dbname == "eval":
            c.executemany("INSERT INTO evaluation VALUES (%d, %s, %s)", entries_3rd)
        elif self.dbname == "registration":
            c.executemany("INSERT INTO student VALUES (%d, %s, %s)", entries_4th)
        
        conn.commit()
        conn.close()
        return

In [13]:
print("dump course/class...")
mymssqldbprocess_edu = mssqldbprocess_edu("192.168.7.155", "SA", "Helios123", "course")
mymssqldbprocess_edu.createedu()
batch_size = 50
for i in range(0, 2):
    mymssqldbprocess_edu.filledu(i*batch_size , (i+1)*batch_size)

print("dump eval/evaluation...")
mymssqldbprocess_edu = mssqldbprocess_edu("192.168.7.155", "SA", "Helios123", "eval")
mymssqldbprocess_edu.createedu()
batch_size = 50
for i in range(0, 2):
    mymssqldbprocess_edu.filledu(i*batch_size , (i+1)*batch_size)

print("dump registration/student...")
mymssqldbprocess_edu = mssqldbprocess_edu("192.168.7.155", "SA", "Helios123", "registration")
mymssqldbprocess_edu.createedu()
batch_size = 50
for i in range(0, 2):
    mymssqldbprocess_edu.filledu(i*batch_size , (i+1)*batch_size)

dump course/class...
dump eval/evaluation...
dump registration/student...


### End of file